In [1]:
import pandas as pd
from deloitte import Deloitte

In [2]:
data = Deloitte("MA_Exer_PikesPeak_Females.txt")
#data = Deloitte("MA_Exer_PikesPeak_Males.txt")
data.clean_data()


ValueError: no valid date format found

## Code Drafting for deloitte package development:

In [3]:
file_names = ["MA_Exer_PikesPeak_Females.txt", "MA_Exer_PikesPeak_Males.txt"]
test_df = pd.read_csv('data/raw/{}'.format(file_names[0]),
                     encoding='latin-1', sep='\t')

In [4]:
# renaming column names
test_df.columns = map(str.lower, test_df.columns)
test_df.rename(columns={'div/tot':'div_total', 'ag':'age','gun tim':'gun_time', 'net tim':'net_time'}, inplace=True)
clean_cols = test_df.columns.tolist()
clean_cols.pop(3) # removes `name` column
clean_cols.pop(4) # removes `hometown` column

print(clean_cols)
# cleaning special symbols from columns to normalize data
for col in clean_cols:
    test_df[col].replace(to_replace='[#*^a-zA-Z ]',value='', regex=True, inplace=True)
test_df['hometown'].replace(to_replace='[,.]', value='', regex=True, inplace=True)

test_df['gun_time']


['place', 'div_total', 'num', 'age', 'gun_time', 'net_time', 'pace']


0         32:59
1         33:14
2         34:26
3         34:43
4         35:04
         ...   
1100    1:43:47
1101    1:50:31
1102    1:50:31
1103    1:46:49
1104    1:46:49
Name: gun_time, Length: 1105, dtype: object

In [5]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%H:%M:%S', '%M:%S', ':%S'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

                
test_df['gun_time'] = test_df['gun_time'].map(try_parsing_date)
test_df['gun_time'] = test_df['gun_time'] - datetime(1900, 1, 1) # Remove date part
test_df['gun_time'].dt.total_seconds()

test_df['net_time'] = test_df['net_time'].map(try_parsing_date)
test_df['net_time'] = test_df['net_time'] - datetime(1900, 1, 1) # Remove date part
test_df['net_time'].dt.total_seconds()

test_df['pace'] = test_df['pace'].map(try_parsing_date)
test_df['pace'] = test_df['pace'] - datetime(1900, 1, 1) # Remove date part
test_df['pace'].dt.total_seconds()

#pd.to_datetime(test_df.gun_time, format='%H:%M:%S.%f')


0        319.0
1        321.0
2        333.0
3        335.0
4        339.0
         ...  
1100     972.0
1101    1018.0
1102    1018.0
1103    1032.0
1104    1032.0
Name: pace, Length: 1105, dtype: float64

In [6]:
def rreplace(s, old=' ', new=', ', occurrence=1):
    li = s.rsplit(old, occurrence)
    return new.join(li)

test_df['hometown']=test_df['hometown'].map(rreplace)

test_df[['city','state']]=test_df.hometown.str.split(',', expand=True)
test_df['state'].replace(to_replace=' ', value='', regex=True, inplace=True)

In [ ]:
#test_df.loc[3,'state']

In [7]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
print(abbrev_to_us_state)
test_df['state']=test_df['state'].map(abbrev_to_us_state)

{'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia', 'AS': 'American Samoa', 'GU': 'Guam', 'MP': 'No

In [8]:
set(test_df.loc[test_df['state'].isnull(),:]['city'])

# Filling in due to Experience 
missing_states={'Ellicott City':'Maryland', 'Fredericksburg':'Virginia', 
                'North Potomac':'Maryland', 'Silver Spring':'Maryland',
               'Washington':'District of Columbia'}
print(missing_states.keys())
subset = test_df.loc[test_df['city'].isin(missing_states.keys()),'city']
test_df.loc[subset.index,'state'] =\
      test_df.loc[subset.index,'city'].map(missing_states)


test_df.loc[test_df['state'].isnull(), :].shape

dict_keys(['Ellicott City', 'Fredericksburg', 'North Potomac', 'Silver Spring', 'Washington'])


(26, 11)

In [9]:
test_df.loc[test_df['age'].isna(), :]
test_df.loc[test_df['age']==0, :]
test_df.loc[test_df['age']<0, :]

,place,div_total,num,name,age,hometown,gun_time,net_time,pace,city,state
382,383,4/15,1917,Stefanie Merritt,-1.0,"Alexandria, VA",00:55:33,00:54:22,00:08:45,Alexandria,Virginia
1008,1009,15/15,1778,Susan Mackey,-1.0,"North Potomac, M",01:16:36,01:11:18,00:11:29,North Potomac,Maryland


In [10]:
import numpy as np
def division_parser(x):
    if (pd.isnull(x) or x<0):
        return np.nan
    elif (x>0) and (x<=14):
        return 1
    elif (x>=15) and (x<=19):
        return 2
    else:
        return int(x/10)
    
    
test_df['division_new'] = test_df['age'].map(division_parser)

In [ ]:
test_df.loc[~test_df['division_new'].isnull(), :]